# Evaluating effects of FSL TOPUP and EPIC
..on individual MNI regions provided by Neuromorphometrics, Inc. (http://Neuromorphometrics.com/)
# Part 1: Running all corrections.

Printing settings

In [1]:
# Run once to store stdout
import sys
nb_stdout = sys.stdout

In [2]:
# Redirect stdout to console, to not get too much text output in the notebook
# This means that the notebook will not output any text. Text will be redirected
# to the terminal where the notebook was started.
sys.stdout = open(1, 'w')

In [3]:
# A print test
print("Hei!")

In [4]:
# Get printing back to the notebook if preferred
# Run once
#sys.stdout = nb_stdout

### __Procedure leading up to this assessment__:

Relative to the location of main.py (not the notebook), specify parameters of the main program run:

In [4]:
DICOM_directory = "../DICOM_no_spaces"
output_directory_suffix = "2019_04_25"

The corrections output base directory from main.py is then in this toturial (relative path from main.py directory):

In [5]:
corrections_base_directory = "../epi_corrections_out_" + output_directory_suffix
corrections_base_directory

'../epi_corrections_out_2019_04_25'

Get the (full) path to the epi_corrections root directory:

In [6]:
from os import getcwd
from os.path import abspath
epi_corrections_root_directory = abspath(getcwd() + "/..") # Go one directory up from current working directory
epi_corrections_root_directory

'/media/loek/HDD3TB1/data/IVS_EPI_BASELINE/epi_corrections'

0. Verify that epi_corrections successfully detects EPI pairs. This will also perform the dicom -> nifti conversion.

In [41]:
%%bash -s "$epi_corrections_root_directory" "$DICOM_directory" "$output_directory_suffix" "$corrections_base_directory"
epi_corrections_root_directory=$1
DICOM_directory=$2
output_directory_suffix=$3
corrections_base_directory=$4
cd $epi_corrections_root_directory
mkdir -p $corrections_base_directory
pipeline_report_file=$corrections_base_directory/pipeline_report_$output_directory_suffix.txt
full_command='python3 main.py 
                    --DICOM_directory='"'$DICOM_directory'"'
                    --output_directory_suffix='"'$output_directory_suffix'"'
                    --run_dcm2niix
                    --print_epi_pairs
                    2>&1 | tee $pipeline_report_file'
eval $full_command

See the pipeline reports:

corrections_base_directory/pipeline_report_*.txt

corrections_base_directory/EPI/DICOM to NIFTI conversion report with keyword epi.txt

corrections_base_directory/FLAIR_3D/DICOM to NIFTI conversion report with keyword flair_3d.txt

for logs of the dcm2niix conversions.

stdout + stderr for the run preparation script scripts/EPI_hmc_nICE_prepare_dir.sh
is not saved to a file.

__Inspect__

corrections_base_directory/pipeline_report_*.txt

to verify that __blip-down, blip-up EPI pairs__ have been successfully identified.

#### Nordic ICE (nICE) Batch Head Motion Correction (BHMC):
1. epi_corrections is used to create an EPI directory for nICE BHMC only. It will use scripts/EPI_hmc_nICE_prepare_dir.sh

In [42]:
%%bash -s "$epi_corrections_root_directory" "$DICOM_directory" "$output_directory_suffix" "$corrections_base_directory"
epi_corrections_root_directory=$1
DICOM_directory=$2
output_directory_suffix=$3
corrections_base_directory=$4
cd $epi_corrections_root_directory
mkdir -p $corrections_base_directory
pipeline_report_file=$corrections_base_directory/pipeline_report_$output_directory_suffix.txt
full_command='python3 main.py 
                    --DICOM_directory='"'$DICOM_directory'"'
                    --output_directory_suffix='"'$output_directory_suffix'"'
                    --epi_hmc
                    --epi_nICE_hmc_not_done
                    2>&1 | tee -a $pipeline_report_file'
eval $full_command

2. nICE BHMC is run on the prepared directory. See scripts/nICE_batch_hmc_example.PNG
for the necessary settings:

![title](../scripts/nICE_batch_hmc_example.PNG)

#### FSL TOPUP and EPIC intensity and distortion corrections:
3. epi_corrections is run completely, performing FSL TOPUP and EPIC corrections on the BHMC epi images. It will first use scripts/EPI_hmc_nICE_prepare_copy_back.sh to get the BHMC EPIs. After performing FSL TOPUP and EPIC corrections, respectively, it will perform FreeSurfer mri_robust_register to a structural image (FLAIR 3D) of EPI images, and assess similarities between combinations of corrected and non-corrected EPIs, using Normalized Mutual Information. Results should be in `<corrections_base_directory>`/TOPUP/topup_performance_metrics.txt and `<corrections_base_directory>`/EPIC/epic_performance_metrics.txt .

In [43]:
%%bash -s "$epi_corrections_root_directory" "$DICOM_directory" "$output_directory_suffix" "$corrections_base_directory"
epi_corrections_root_directory=$1
DICOM_directory=$2
output_directory_suffix=$3
corrections_base_directory=$4
cd $epi_corrections_root_directory
pipeline_report_file=$corrections_base_directory/pipeline_report_$output_directory_suffix.txt
full_command='python3 main.py 
                    --DICOM_directory='"'$DICOM_directory'"'
                    --output_directory_suffix='"'$output_directory_suffix'"'
                    --epi_hmc
                    --run_topup
                    --run_epic
                    2>&1 | tee -a $pipeline_report_file'
eval $full_command

See the appended pipeline report:

corrections_base_directory/pipeline_report_*.txt

for correction logs.

To see what's happening during the run, you can follow the changes of the file by running the terminal command from corrections_base_directory:

tail -f pipeline_report_*.txt


#### nICE Batch Perfusion Analysis (BPA) on uncorrected, TOPUP corrected and EPIC corrected EPIs:
4. ../scripts/EPI_perfusion_anal_nICE_make_uncorr_dir.sh is run to make a directory containing uncorrected EPIs of interest (thus containing the DSC EPIs only and not also the opposite phase encoded EPIs):

In [58]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_directory"
epi_corrections_root_directory=$1
corrections_base_directory=$2
cd $epi_corrections_root_directory
full_command='bash scripts/EPI_perfusion_anal_nICE_make_uncorr_dir.sh 
            "'$corrections_base_directory'/EPI"'
eval $full_command

The raw DSC should now be copied into the directory

corrections_base_directory/EPI_raw_DSC

for easier raw DSC Perfusion analysis.

5. ../scripts/EPI_perfusion_anal_nICE_prepare_dir.sh is run three times to prepare directories for nICE BPA:

In [59]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_directory"
epi_corrections_root_directory=$1
corrections_base_directory=$2
cd $epi_corrections_root_directory

command_raw='bash scripts/EPI_perfusion_anal_nICE_prepare_dir.sh 
            "'$corrections_base_directory'/EPI_raw_DSC" raw'
eval $command_raw

command_topup='bash scripts/EPI_perfusion_anal_nICE_prepare_dir.sh 
            "'$corrections_base_directory'/EPI_applytopup" topup'
eval $command_topup

command_epic='bash scripts/EPI_perfusion_anal_nICE_prepare_dir.sh 
            "'$corrections_base_directory'/EPI_applyepic" epic'
eval $command_epic

A directory suited for NordicICE Batch Perfusion Analysis should now have been created:

corrections_base_directory/EPI_for_nICE_batch_perfusion_analysis

that contains raw, topup and epic corrected DSC files.

6. nICE BPA is run. Let most of the settings to default settings. See scripts/nICE batch_perfusion_example.PNG . The only change in Main settings from this nICE version (4.0) was to remove the text "prefix_" in the "Output image prefix:" box. Only rCBV is analysed further, so only rCBV needs to be ticked. Under AIF, check off No AIF. Under option, under Normalization tick Normalize CBV and CBF. Under Leakage correction, tick Apply leakage (extravasation) correction. Under Signal conversion, Signal conversion type is "SI to delR2".

![title](../scripts/nICE_batch_perfusion_example.PNG)

nICE should now have created a Perf folder within each of the random number named folders in EPI_for_nICE_batch_perfusion_analysis .

7. The commands in the following cell copy back images in Perf folders to repsective *perf folders within EPI_raw_DSC, EPI_applytopup and EPI_applyepic .

In [60]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_directory"
epi_corrections_root_director=$1
corrections_base_directory=$2
cd $epi_corrections_root_director

command_raw='bash scripts/EPI_perfusion_anal_nICE_prepare_copy_back.sh 
            "'$corrections_base_directory'/EPI_raw_DSC"'
eval $command_raw

command_topup='bash scripts/EPI_perfusion_anal_nICE_prepare_copy_back.sh 
            "'$corrections_base_directory'/EPI_applytopup"'
eval $command_topup

command_epic='bash scripts/EPI_perfusion_anal_nICE_prepare_copy_back.sh 
            "'$corrections_base_directory'/EPI_applyepic"'
eval $command_epic

#### Matlab SPM coregistration + MNI normalization
8. som_pipeline/coreg_norm_images.m is run to perform coregistration (to FLAIR 3D) (again, but now using SPM) and MNI normalization of raw, TOPUP and EPIC corrected EPIs, off-resonance fields (TOPUP), displacement fields (EPIC), as well as associated Normalized Relative CBV maps from BPA (not the the orther Perfusion images). Verify that epi_corrections_out_dir in ../spm_pipeline/coreg_norm_images.m is set to the correct directory then run:

In [11]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_directory" "$output_directory_suffix"
epi_corrections_root_directory=$1
corrections_base_directory=../$2
output_directory_suffix=$3

spm_path=/media/loek/HDD3TB1/apps/spm12 # This needs to be changed
cd $epi_corrections_root_directory
pipeline_report_file=$2/pipeline_report_spm_$output_directory_suffix.txt
run_command='matlab 
                -nodisplay 
                -nosplash 
                -nodesktop 
                -r "cd('"'spm_pipeline'"'); 
                    addpath('"'$spm_path'"'); 
                    myCluster = parcluster; 
                    myCluster.NumWorkers = 6; 
                    saveProfile(myCluster); 
                    parpool(6); 
                    coreg_norm_images('"'$corrections_base_directory'"'); 
                    exit;" 
                2>&1 | tee -a $pipeline_report_file'
eval $run_command

To see the log, run from corrections_base_directory

tail -f pipeline_report_spm*.txt

https://se.mathworks.com/matlabcentral/answers/72661-starting-multiple-workers-from-the-command-prompt

https://se.mathworks.com/help/parallel-computing/run-code-on-parallel-pools.html

#### Calculate MNI CBV region histograms
9. Run:

In [24]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_directory" "$output_directory_suffix"
epi_corrections_root_directory=$1
corrections_base_directory=../$2
output_directory_suffix=$3
cd $epi_corrections_root_directory
pipeline_report_file=$2/pipeline_report_spm_$output_directory_suffix.txt
run_command='matlab 
                -nojvm 
                -nodisplay 
                -nosplash 
                -nodesktop 
                -r "cd('"'spm_pipeline'"'); 
                    compute_and_save_CBV_region_histograms('"'$corrections_base_directory'"'); 
                    exit;" 
                2>&1 | tee -a $pipeline_report_file'
eval $run_command

To see the log, run from corrections_base_directory

tail -f pipeline_report_spm*.txt

__NOTE:__ This command assumes that there exist a (resliced to DSC voxel resolution) rlabels_Neuromorphometrics.nii in corrections_base_directory from previous step (8.)

Matlab .mat files of MNI region histograms should now be saved to `<corrections_base_directory>`/CBV_histograms .

http://mmcdan.github.io/posts/interacting-with-the-shell-via-jupyter-notebook/